In [1]:
import pandas as pd
import numpy as np

from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [2]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1MINUTE,
                                      str(3000) + ' minutes ago UTC'))

# First 6 column represent Time and OHLCV values
df = df.iloc[:, :6]
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
for column in df.columns:
    if column != 'Time':
        df[column] = df[column].astype(float)

# Set Time as index
df.set_index('Time', inplace=True)

# Generate Technical Indicators

In [3]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2024-07-09 08:57:00,57664.00,57667.25,57623.80,57623.81,12.43372
2024-07-09 08:58:00,57623.81,57634.13,57594.24,57604.01,24.52810
2024-07-09 08:59:00,57604.01,57615.54,57593.74,57615.54,7.30705
2024-07-09 09:00:00,57615.54,57648.51,57580.00,57598.02,23.64362
2024-07-09 09:01:00,57598.02,57634.00,57588.00,57620.01,11.78448
...,...,...,...,...,...
2024-07-11 10:52:00,58395.99,58426.57,58390.00,58395.50,12.88638
2024-07-11 10:53:00,58395.49,58395.50,58380.03,58380.03,6.08716
2024-07-11 10:54:00,58380.04,58380.04,58374.00,58374.01,5.41815


In [4]:
df['SMA50'] = df['Close'].rolling(50).mean()
df['SMA200'] = df['Close'].rolling(200).mean()
df['RSI'] = momentum.rsi(df['Close'])
df['MACD'] = trend.macd(df['Close'])
df['MACD_sig'] = trend.macd_signal(df['Close'])
df['MACD_diff'] = trend.macd_diff(df['Close'])
df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])


# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [5]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2024-07-09 12:16:00,57533.99,57544.00,57520.06,57520.07,5.23758,57466.0200,57495.89915,60.362256,17.343489,5.581234,11.762255,0.000434,20.411896,14.902392,31.034067
2024-07-09 12:17:00,57520.07,57520.07,57501.04,57513.08,3.96235,57467.3614,57495.34550,58.856302,18.584813,8.181950,10.402863,0.000895,20.595399,18.592369,29.687546
2024-07-09 12:18:00,57513.09,57513.09,57480.00,57493.99,11.62696,57467.4412,57494.79540,54.832835,17.822718,10.110103,7.712615,0.001259,19.916692,21.973210,27.456719
2024-07-09 12:19:00,57494.00,57494.00,57455.09,57455.10,3.35076,57465.7358,57493.99320,47.681686,13.920192,10.872121,3.048071,0.000577,18.572341,25.626568,25.071035
2024-07-09 12:20:00,57455.09,57484.00,57453.15,57484.00,4.29596,57464.6556,57493.42310,52.626141,13.009435,11.299584,1.709851,0.001114,17.388433,24.290972,23.339489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-11 10:52:00,58395.99,58426.57,58390.00,58395.50,12.88638,58313.4288,58223.80950,60.367459,24.016869,25.930551,-1.913682,0.002065,32.533851,11.351776,35.129858
2024-07-11 10:53:00,58395.49,58395.50,58380.03,58380.03,6.08716,58311.5492,58224.91965,56.051672,22.283180,25.201077,-2.917897,0.002041,33.248027,13.571802,33.661011
2024-07-11 10:54:00,58380.04,58380.04,58374.00,58374.01,5.41815,58311.3344,58225.93510,54.421190,20.190709,24.199003,-4.008294,0.002026,33.546430,15.062142,33.079458


In [6]:
#SMA Golden Cross & Death Cross
sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))

In [7]:
# MACD Below 0 Cross
macd_buy_cond = (df['MACD_diff'] > 0) & (df['MACD_diff'].shift(1) < 0) & (df['MACD'] < 0) &(df['MACD_sig'] < 0)
macd_sell_cond = (df['MACD_diff'] < 0) & (df['MACD_diff'].shift(1) > 0) & (df['MACD'] > 0) &(df['MACD_sig'] > 0)
df['Signal_macd'] = np.where(macd_buy_cond, 1, np.where(macd_sell_cond, -1, 0))

In [8]:
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd
Time,,,,,,,,,,,,,,,,,
2024-07-09 12:16:00,57533.99,57544.00,57520.06,57520.07,5.23758,57466.0200,57495.89915,60.362256,17.343489,5.581234,11.762255,0.000434,20.411896,14.902392,31.034067,0,0
2024-07-09 12:17:00,57520.07,57520.07,57501.04,57513.08,3.96235,57467.3614,57495.34550,58.856302,18.584813,8.181950,10.402863,0.000895,20.595399,18.592369,29.687546,0,0
2024-07-09 12:18:00,57513.09,57513.09,57480.00,57493.99,11.62696,57467.4412,57494.79540,54.832835,17.822718,10.110103,7.712615,0.001259,19.916692,21.973210,27.456719,0,0
2024-07-09 12:19:00,57494.00,57494.00,57455.09,57455.10,3.35076,57465.7358,57493.99320,47.681686,13.920192,10.872121,3.048071,0.000577,18.572341,25.626568,25.071035,0,0
2024-07-09 12:20:00,57455.09,57484.00,57453.15,57484.00,4.29596,57464.6556,57493.42310,52.626141,13.009435,11.299584,1.709851,0.001114,17.388433,24.290972,23.339489,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-11 10:52:00,58395.99,58426.57,58390.00,58395.50,12.88638,58313.4288,58223.80950,60.367459,24.016869,25.930551,-1.913682,0.002065,32.533851,11.351776,35.129858,0,0
2024-07-11 10:53:00,58395.49,58395.50,58380.03,58380.03,6.08716,58311.5492,58224.91965,56.051672,22.283180,25.201077,-2.917897,0.002041,33.248027,13.571802,33.661011,0,0
2024-07-11 10:54:00,58380.04,58380.04,58374.00,58374.01,5.41815,58311.3344,58225.93510,54.421190,20.190709,24.199003,-4.008294,0.002026,33.546430,15.062142,33.079458,0,0
